# Analyze A/B Test Result

This project will assure you have mastered the subjects covered in the statistics lessons. The hope is to have this project be as comprehensive of these topics as possible. Good luck!

## Table of Contents

- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)


<a id='intro'></a>
### Introduction

A/B tests are very commonly performed by data analysts and data scientists.  It is important that you get some practice working with the difficulties of these 

For this project, you will be working to understand the results of an A/B test run by an e-commerce website.  Your goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.

<a id='probability'></a>
#### Part I - Probability

To get started, let's import our libraries.


In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)

`1.` Now, read in the `ab_data.csv` data. Store it in `df`.  **Use your dataframe to answer the questions in Quiz 1 of the classroom.**

a. Read in the dataset and take a look at the top few rows here:

In [2]:
df = pd.read_csv('ab_data.csv')
df.head()

user_id                   timestamp      group landing_page  converted
0   851104  2017-01-21 22:11:48.556739    control     old_page          0
1   804228  2017-01-12 08:01:45.159739    control     old_page          0
2   661590  2017-01-11 16:55:06.154213  treatment     new_page          0
3   853541  2017-01-08 18:28:03.143765  treatment     new_page          0
4   864975  2017-01-21 01:52:26.210827    control     old_page          1

b. Use the cell below to find the number of rows in the dataset.

In [7]:
#show the total number of rows
df.shape[0]

294478

c. The number of unique users in the dataset.

In [8]:
#calculare the number of unique users
df.nunique()

user_id         290584
timestamp       294478
group                2
landing_page         2
converted            2
dtype: int64

d. The proportion of users converted.

In [9]:
#calculate the converted users
df['converted'].mean()

0.11965919355605512

e. The number of times the `new_page` and `treatment` don't match.

In [10]:
df_AB = len(df.query('group == "treatment" and landing_page != "new_page"')) + \
                len(df.query('group != "treatment" and landing_page == "new_page"'))

#calculate thenumber of time new_page and treatment
df_AB

3893

f. Do any of the rows have missing values?

In [11]:
#view if there is any missing value
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


`2.` For the rows where **treatment** does not match with **new_page** or **control** does not match with **old_page**, we cannot be sure if this row truly received the new or old page.  Use **Quiz 2** in the classroom to figure out how we should handle these rows.  

a. Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz.  Store your new dataframe in **df2**.

In [14]:
#remove the mismatch data
df2 = df.drop(df[((df.landing_page == 'new_page') & (df.group != 'treatment')) | ((df.landing_page != 'new_page') & (df.group == 'treatment'))].index)

In [15]:
# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

0

`3.` Use **df2** and the cells below to answer questions for **Quiz3** in the classroom.

a. How many unique **user_id**s are in **df2**?

In [16]:
# check for info
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290585 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       290585 non-null  int64 
 1   timestamp     290585 non-null  object
 2   group         290585 non-null  object
 3   landing_page  290585 non-null  object
 4   converted     290585 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 13.3+ MB


In [17]:
#calculare the number of unique user_id 
len(df2['user_id'].unique())

290584

b. There is one **user_id** repeated in **df2**.  What is it?

In [18]:
#checking for the duplicate user_id
df2[df2.duplicated(['user_id'], keep=False)]

user_id                   timestamp      group landing_page  converted
1899   773192  2017-01-09 05:37:58.781806  treatment     new_page          0
2893   773192  2017-01-14 02:55:59.590927  treatment     new_page          0

c. What is the row information for the repeat user_id?

rows 1899 and 2893 are repeated.

d. Remove **one** of the rows with a duplicate **user_id**, but keep your dataframe as **df2**.

In [19]:
# removing one of the duplicated rows
df2.drop(labels=2893, inplace=True)

In [20]:
#checking for the rows have been drop
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290584 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       290584 non-null  int64 
 1   timestamp     290584 non-null  object
 2   group         290584 non-null  object
 3   landing_page  290584 non-null  object
 4   converted     290584 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 13.3+ MB


`4.` Use **df2** in the cells below to answer the quiz questions related to **Quiz 4** in the classroom.

a. What is the probability of an individual converting regardless of the page they receive?

In [21]:
#calculate the average of the converted column
df2['converted'].mean()

0.11959708724499628

b. Given that an individual was in the `control` group, what is the probability they converted?

In [22]:
#checking the probability of converted column where the value is control
df2.converted[df2.group == 'control'].mean()

0.1203863045004612

c. Given that an individual was in the `treatment` group, what is the probability they converted?

In [23]:
#checking the probability of converted column where the value is treatment
df2.converted[df2.group == 'treatment'].mean()

0.11880806551510564

d. What is the probability that an individual received the new page?

In [24]:
df2[df2.landing_page == 'new_page'].count()/df2.shape[0]

user_id         0.500062
timestamp       0.500062
group           0.500062
landing_page    0.500062
converted       0.500062
dtype: float64

e. Consider your results from parts (a) through (d) above, and explain below whether you think there is sufficient evidence to conclude that the new treatment page leads to more conversions.

* Given the probability that an individual received the new page is 0.50006, it suggests that there is an equal chance that an individual received the old page at the same time.
* Besides, the probability of conversion from the control group and treatment group is almost similar. Just different at the 3rd decimal point.
* The percentage of converted in the control group is 12.04% and converted in the treatment group is 11.88%.
* The difference is very subtle so I do not believe there is a signifcant enough difference for a certain page that will convert more users.

<a id='ab_test'></a>
### Part II - A/B Test

Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.  

However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


`1.` For now, consider you need to make the decision just based on all the data provided.  If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should your null and alternative hypotheses be?  You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the converted rates for the old and new pages.

#### Null Hypotheses: if p-value is less than 5%, the old page has a higher chance of converting users.
#### Alternative Hypotheses: if p-value is equal to or greater than 5%, then the new page has a higher chance of converting users.

`2.` Assume under the null hypothesis, $p_{new}$ and $p_{old}$ both have "true" success rates equal to the **converted** success rate regardless of page - that is $p_{new}$ and $p_{old}$ are equal. Furthermore, assume they are equal to the **converted** rate in **ab_data.csv** regardless of the page. <br><br>

Use a sample size for each page equal to the ones in **ab_data.csv**.  <br><br>

Perform the sampling distribution for the difference in **converted** between the two pages over 10,000 iterations of calculating an estimate from the null.  <br><br>

Use the cells below to provide the necessary parts of this simulation.  If this doesn't make complete sense right now, don't worry - you are going to work through the problems below to complete this problem.  You can use **Quiz 5** in the classroom to make sure you are on the right track.<br><br>

a. What is the **conversion rate** for $p_{new}$ under the null?

In [25]:
#The Null hypothesis states there is no difference between the 
#conversion rates of an old and new page. Thus, the whole dataset is used to 
#calculate the conversion rate for the new page

p_new = df2.converted.mean()
p_new

0.11959708724499628

b. What is the **conversion rate** for $p_{old}$ under the null? <br><br>

In [26]:
#calculate the conversion rate of old page
p_old = df2.converted.mean()
p_old

0.11959708724499628

c. What is $n_{new}$, the number of individuals in the treatment group?

In [27]:
#checking the number of users who got a new page for group treatment
n_new = df2[df2.group == 'treatment'].count()[0]
n_new

145310

d. What is $n_{old}$, the number of individuals in the control group?

In [28]:
#checking the number of users who remained with an old page for group control
n_old = df2[df2.group == 'control'].count()[0]
n_old

145274

e. Simulate $n_{new}$ transactions with a conversion rate of $p_{new}$ under the null.  Store these $n_{new}$ 1's and 0's in **new_page_converted**.

In [29]:
#simulating new transactions that have the probability of recieving a new page
new_page_converted = np.random.binomial(1,p_new,n_new) 
new_page_converted.mean()

0.11930355791067374

f. Simulate $n_{old}$ transactions with a conversion rate of $p_{old}$ under the null.  Store these $n_{old}$ 1's and 0's in **old_page_converted**.

In [38]:
#simulating new transactions that have the probability of having an old page
old_page_converted = np.random.binomial(1,p_old,n_old)
old_page_converted.mean()

0.11838319313848314

g. Find $p_{new}$ - $p_{old}$ for your simulated values from part (e) and (f).

In [39]:
#checking the probability of new and old pages
p_new = new_page_converted.mean()
p_old = old_page_converted.mean()
p_new - p_old

0.0009203647721905961

h. Create 10,000 $p_{new}$ - $p_{old}$ values using the same simulation process you used in parts (a) through (g) above. Store all 10,000 values in a NumPy array called **p_diffs**.

In [40]:
p_diffs = []
for _ in range(10000):
    new_page_converted = np.random.binomial(1,p_new,n_new)
    old_page_converted = np.random.binomial(1,p_old,n_old)
    p_diffs.append(new_page_converted.mean() - old_page_converted.mean())

i. Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.

In [41]:
# convert p_diffs to numpy array for array based computations in the future
p_diffs = np.array(p_diffs)

# plotting a histogram
plt.hist(p_diffs)
plt.grid()
plt.axvline(p_diffs.mean(), color='r', label='mean')
plt.legend();

j. What proportion of the **p_diffs** are greater than the actual difference observed in **ab_data.csv**?

In [42]:
# calculate actual difference observed
actual_diff = df2.converted[df2.group == 'treatment'].mean() - df2.converted[df2.group == 'control'].mean()
(actual_diff < p_diffs).mean()

0.9821

k. Please explain using the vocabulary you've learned in this course what you just computed in part **j.** What is this value called in scientific studies?  What does this value mean in terms of whether or not there is a difference between the new and old pages?

* What we computed in part **j** is called p-value in scientific studies.
* P-value is the probability of observing your statistic if the null hypothesis is true.
* In our case, the p-value is so significant that we can confidently say that accept the null hypothesis.


l. We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. Fill in the below to calculate the number of conversions for each page, as well as the number of individuals who received each page. Let `n_old` and `n_new` refer the the number of rows associated with the old page and new pages, respectively.

In [43]:
# checking before doing the further process
df2.head()

user_id                   timestamp      group landing_page  converted
0   851104  2017-01-21 22:11:48.556739    control     old_page          0
1   804228  2017-01-12 08:01:45.159739    control     old_page          0
2   661590  2017-01-11 16:55:06.154213  treatment     new_page          0
3   853541  2017-01-08 18:28:03.143765  treatment     new_page          0
4   864975  2017-01-21 01:52:26.210827    control     old_page          1

In [44]:
import statsmodels.api as sm

convert_old = df2[df2.group == 'control'].converted.sum()
convert_new = df2[df2.group == 'treatment'].converted.sum()
n_old = df2[df2.group == 'control'].converted.count()
n_new = df2[df2.group == 'treatment'].converted.count()

m. Now use `stats.proportions_ztest` to compute your test statistic and p-value.  [Here](https://docs.w3cub.com/statsmodels/generated/statsmodels.stats.proportion.proportions_ztest/) is a helpful link on using the built in.

From the link, we have all the values required. However, it is worth noticing that the argument alternative in the function is 'two-sided' by default. But our hypothesis is not designed around the 'either-or' scenario at this point. Therefore, the argument alternative will be 'smaller, meaning one-tailed experiment.

In [45]:
# based upon the given documentation
z_score, p_value = sm.stats.proportions_ztest([convert_old, convert_new], [n_old, n_new], alternative='smaller')
z_score, p_value

(1.3109241984234394, 0.9050583127590245)

n. What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?

* A z-score represents how many standard deviations away our data point is from the mean.
* The p-value of 0.9050 is very close to the p-value we computed earlier in part j.
* With this computation, we can indeed say we accept the null hypothesis.

<a id='regression'></a>
### Part III - A regression approach

`1.` In this final part, you will see that the result you achieved in the A/B test in Part II above can also be achieved by performing regression.<br><br> 

a. Since each row is either a conversion or no conversion, what type of regression should you be performing in this case?

#### This is the case of Logistic Regression. In this consequence, we want to predict something that has only two possible outcomes.

b. The goal is to use **statsmodels** to fit the regression model you specified in part **a.** to see if there is a significant difference in conversion based on which page a customer receives. However, you first need to create in df2 a column for the intercept, and create a dummy variable column for which page each user received.  Add an **intercept** column, as well as an **ab_page** column, which is 1 when an individual receives the **treatment** and 0 if **control**.

In [46]:
df2[['control','treatment']]= pd.get_dummies(df2['group'])
df2 = df2.drop('control',axis = 1)

df2.head()

user_id                   timestamp      group landing_page  converted  \
0   851104  2017-01-21 22:11:48.556739    control     old_page          0   
1   804228  2017-01-12 08:01:45.159739    control     old_page          0   
2   661590  2017-01-11 16:55:06.154213  treatment     new_page          0   
3   853541  2017-01-08 18:28:03.143765  treatment     new_page          0   
4   864975  2017-01-21 01:52:26.210827    control     old_page          1   

   treatment  
0          0  
1          0  
2          1  
3          1  
4          0

In [47]:
df3 = df2.rename(columns={'treatment': 'ab_page'})
df3.head()

user_id                   timestamp      group landing_page  converted  \
0   851104  2017-01-21 22:11:48.556739    control     old_page          0   
1   804228  2017-01-12 08:01:45.159739    control     old_page          0   
2   661590  2017-01-11 16:55:06.154213  treatment     new_page          0   
3   853541  2017-01-08 18:28:03.143765  treatment     new_page          0   
4   864975  2017-01-21 01:52:26.210827    control     old_page          1   

   ab_page  
0        0  
1        0  
2        1  
3        1  
4        0

c. Use statsmodels to import your regression model. Instantiate the model, and fit the model using the two columns you created in part b. to predict whether or not an individual converts.

In [48]:
from scipy import stats
stats.chisqprob = lambda chisq, df3: stats.chi2.sf(chisq, df3)

df3['intercept'] = 1

lm = sm.Logit(df3['converted'],df3[['intercept','ab_page']])
results = lm.fit()

Optimization terminated successfully.
         Current function value: 0.366118
         Iterations 6


d. Provide the summary of your model below, and use it as necessary to answer the following questions.

In [50]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              converted   No. Observations:               290584
Model:                          Logit   Df Residuals:                   290582
Method:                           MLE   Df Model:                            1
Date:                Mon, 23 Aug 2021   Pseudo R-squ.:               8.077e-06
Time:                        13:11:39   Log-Likelihood:            -1.0639e+05
converged:                       True   LL-Null:                   -1.0639e+05
Covariance Type:            nonrobust   LLR p-value:                    0.1899
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -1.9888      0.008   -246.669      0.000      -2.005      -1.973
ab_page       -0.0150      0.011     -1.311      0.190      -0.037       0.007
==============================================================================
"""

e. What is the p-value associated with **ab_page**? Why does it differ from the value you found in **Part II**?<br><br>  **Hint**: What are the null and alternative hypotheses associated with your regression model, and how do they compare to the null and alternative hypotheses in **Part II**?

* As we already built the base for z-score and p-value in part-m, this p-value is different, of course.
* The logistic regression is a two-tailed case. We are exploring only two possible outcomes, and it can go either way.
* An exciting part of observing is the z-score, absolute of which confirms the number in part-m.

  The null and alternative hypotheses associated with the regression model will be as following:

   #### $H_{0}$:$p_{old}$−$p_{new}=0$
 
   #### $H_{1}$:$p_{old}$−$p_{new}≠0$

f. Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

#### There can be many other factors that can be considered to add to our regression model.

* Geographic location is another crucial factor. If the page is available in multiple languages, for example, it can diversify the sample.
* Parameters like click-through rate is another factor to consider.
* A disadvantage of multiple factors in a logistic regression model is that it reduces the power of analysis.


g. Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives in. You will need to read in the **countries.csv** dataset and merge together your datasets on the appropriate rows.  [Here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) are the docs for joining tables. 

Does it appear that country had an impact on conversion?  Don't forget to create dummy variables for these country columns - **Hint: You will need two columns for the three dummy variables.** Provide the statistical output as well as a written response to answer this question.

In [51]:
countries_df = pd.read_csv('./countries.csv')
df_new = countries_df.set_index('user_id').join(df3.set_index('user_id'), how='inner')

In [52]:
df_new.head()

country                   timestamp      group landing_page  \
user_id                                                               
834778       UK  2017-01-14 23:08:43.304998    control     old_page   
928468       US  2017-01-23 14:44:16.387854  treatment     new_page   
822059       UK  2017-01-16 14:04:14.719771  treatment     new_page   
711597       UK  2017-01-22 03:14:24.763511    control     old_page   
710616       UK  2017-01-16 13:14:44.000513  treatment     new_page   

         converted  ab_page  intercept  
user_id                                 
834778           0        0          1  
928468           0        1          1  
822059           1        1          1  
711597           0        0          1  
710616           0        1          1

In [53]:
df_new.head()
df_new.country.unique()

array(['UK', 'US', 'CA'], dtype=object)

In [54]:
#mean of conversion rate
df_new.ab_page.mean()

0.5000619442226688

In [55]:
#mean conversion rate by country
df_new.groupby(['country'], as_index=False).mean()


country  converted   ab_page  intercept
0      CA   0.115318  0.503552        1.0
1      UK   0.120594  0.498247        1.0
2      US   0.119547  0.500459        1.0

In [56]:
#mean conversion rate by landing_page 
df_new.groupby(['ab_page'], as_index=False).mean()

ab_page  converted  intercept
0        0   0.120386        1.0
1        1   0.118808        1.0

In [60]:
# Create the necessary dummy variables
df_new[['CA_ab_page','UK_ab_page', 'US_ab_page']]= pd.get_dummies(df_new['country'])

df_new.head()

country                   timestamp      group landing_page  \
user_id                                                               
834778       UK  2017-01-14 23:08:43.304998    control     old_page   
928468       US  2017-01-23 14:44:16.387854  treatment     new_page   
822059       UK  2017-01-16 14:04:14.719771  treatment     new_page   
711597       UK  2017-01-22 03:14:24.763511    control     old_page   
710616       UK  2017-01-16 13:14:44.000513  treatment     new_page   

         converted  ab_page  intercept  CA  UK  US  CA_ab_page  UK_ab_page  \
user_id                                                                      
834778           0        0          1   0   1   0           0           1   
928468           0        1          1   0   0   1           0           0   
822059           1        1          1   0   1   0           0           1   
711597           0        0          1   0   1   0           0           1   
710616           0        1          1   0   1   0           0           1   

         US_ab_page  
user_id              
834778            0  
928468            1  
822059            0  
711597            0  
710616            0

In [63]:
# fitting the linear model

df_new['intercept'] = 1

lm = sm.Logit(df_new['converted'],df_new[['intercept','ab_page','UK_ab_page','US_ab_page']])
results = lm.fit()

Optimization terminated successfully.
         Current function value: 0.366113
         Iterations 6


In [64]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              converted   No. Observations:               290584
Model:                          Logit   Df Residuals:                   290580
Method:                           MLE   Df Model:                            3
Date:                Mon, 23 Aug 2021   Pseudo R-squ.:               2.323e-05
Time:                        13:20:48   Log-Likelihood:            -1.0639e+05
converged:                       True   LL-Null:                   -1.0639e+05
Covariance Type:            nonrobust   LLR p-value:                    0.1760
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -2.0300      0.027    -76.249      0.000      -2.082      -1.978
ab_page       -0.0149      0.011     -1.307      0.191      -0.037       0.007
UK_ab_page     0.0506      0.028      1.784      0.074      -0.005       0.106
US_ab_page     0.0408      0.027      1.516      0.130      -0.012       0.093
==============================================================================
"""

h. Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if there significant effects on conversion.  Create the necessary additional columns, and fit the new model.  

Provide the summary results, and your conclusions based on the results.

In [65]:
#mean conversion rate by country and landing_page - 
#checking for possible interactions

df_new.groupby(['country','ab_page'], as_index=False).mean()

country  ab_page  converted  intercept  CA  UK  US  CA_ab_page  UK_ab_page  \
0      CA        0   0.118783        1.0   1   0   0           1           0   
1      CA        1   0.111902        1.0   1   0   0           1           0   
2      UK        0   0.120022        1.0   0   1   0           0           1   
3      UK        1   0.121171        1.0   0   1   0           0           1   
4      US        0   0.120630        1.0   0   0   1           0           0   
5      US        1   0.118466        1.0   0   0   1           0           0   

   US_ab_page  
0           0  
1           0  
2           0  
3           0  
4           1  
5           1

In [69]:
df_new.head()
df_new['interaction_uk_ab_page'] = df_new.UK *df_new.ab_page
df_new['interaction_us_ab_page'] = df_new.US *df_new.ab_page

df_new.head()

country                   timestamp      group landing_page  \
user_id                                                               
834778       UK  2017-01-14 23:08:43.304998    control     old_page   
928468       US  2017-01-23 14:44:16.387854  treatment     new_page   
822059       UK  2017-01-16 14:04:14.719771  treatment     new_page   
711597       UK  2017-01-22 03:14:24.763511    control     old_page   
710616       UK  2017-01-16 13:14:44.000513  treatment     new_page   

         converted  ab_page  intercept  CA  UK  US  CA_ab_page  UK_ab_page  \
user_id                                                                      
834778           0        0          1   0   1   0           0           1   
928468           0        1          1   0   0   1           0           0   
822059           1        1          1   0   1   0           0           1   
711597           0        0          1   0   1   0           0           1   
710616           0        1          1   0   1   0           0           1   

         US_ab_page  interaction_uk_ab_page  interaction_us_ab_page  \
user_id                                                               
834778            0                       0                       0   
928468            1                       0                       1   
822059            0                       1                       0   
711597            0                       0                       0   
710616            0                       1                       0   

         uk_ab_page  us_ab_page  
user_id                          
834778            0           0  
928468            0           1  
822059            1           0  
711597            0           0  
710616            1           0

In [72]:
df_new['intercept'] = 1

lm = sm.Logit(df_new['converted'],df_new[['intercept','ab_page','UK','US', 'interaction_uk_ab_page', 'interaction_us_ab_page']])
results = lm.fit()

Optimization terminated successfully.
         Current function value: 0.366109
         Iterations 6


In [73]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              converted   No. Observations:               290584
Model:                          Logit   Df Residuals:                   290578
Method:                           MLE   Df Model:                            5
Date:                Mon, 23 Aug 2021   Pseudo R-squ.:               3.482e-05
Time:                        13:31:30   Log-Likelihood:            -1.0639e+05
converged:                       True   LL-Null:                   -1.0639e+05
Covariance Type:            nonrobust   LLR p-value:                    0.1920
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
intercept                 -2.0040      0.036    -55.008      0.000      -2.075      -1.933
ab_page                   -0.0674      0.052     -1.297      0.195      -0.169       0.034
UK                         0.0118      0.040      0.296      0.767      -0.066       0.090
US                         0.0175      0.038      0.465      0.642      -0.056       0.091
interaction_uk_ab_page     0.0783      0.057      1.378      0.168      -0.033       0.190
interaction_us_ab_page     0.0469      0.054      0.872      0.383      -0.059       0.152
==========================================================================================
"""

In [74]:
np.exp(results.params)

intercept                 0.134794
ab_page                   0.934776
UK                        1.011854
US                        1.017682
interaction_uk_ab_page    1.081428
interaction_us_ab_page    1.048001
dtype: float64

#### Summary: By looking at the odds ratio of the interaction between country and ab_page, we have an equal chance of the ab_page converting users in each country(1.017, 1.012). We thus fail to reject the null hypothesis; the new page does not convert more than the old page.

## Conclusion

Within the framework of this project, we tried to understand whether the company should implement a new page or keep the old page with the following:

* Probability-based approach
* A/B test
* Regression approach

#### Probability based approach:

* We found that probability of an individual receiving the new page is 0.5001
* Meaning, there is almost the same chance that an individual received the old page


#### A/B test:

* In the A/B test, we set up our hypothesis to test if a new page results in better conversion or not.
* We simulated our user groups concerning conversions
* With such a p-value, we failed to reject the null hypothesis
* By using the built-in stats.proportions_ztest, we computed z-score and p-value, which confirmed our earlier p-value and accept the null hypothesis.

#### Regression Approach:

* We looked at exploring two possible outcomes. Whether a new page is better or not.
* By further adding the geographic location of the users, we tried to find if any specific country had an impact on conversion.
* The result gave a similar outlook and suggested that the countries have no impact on the conversion rate.


#### Consideration:

* Due to Change aversion effect, a group of users may give an unfair advantage to the older page.
* Similarly, due to Novelty effect, users may give an unfair advantage to the newer page.